# Zenpli challenge

### Parte 1.2

Imports

In [2]:
import pandas as pd
import numpy as np
import time

Leemos el archivo con Pandas

In [3]:
pd_df = pd.read_csv("file:///content/backend-dev-data-dataset.txt", parse_dates=['date_2'])

Mostramos esquema

In [5]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269339 entries, 0 to 269338
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   key_1    269339 non-null  object        
 1   date_2   269339 non-null  datetime64[ns]
 2   cont_3   269339 non-null  float64       
 3   cont_4   269339 non-null  float64       
 4   disc_5   269339 non-null  int64         
 5   disc_6   269339 non-null  object        
 6   cat_7    269339 non-null  object        
 7   cat_8    269339 non-null  object        
 8   cont_9   269338 non-null  float64       
 9   cont_10  269338 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 20.5+ MB


Pre visualizamos datos

In [37]:
pd_df.head()

,key_1,date_2,cont_3,cont_4,disc_5,disc_6,cat_7,cat_8,cont_9,cont_10
0,HC2030,2016-11-16,622.27,-2.36,2,6,frequent,happy,0.24,0.25
1,sP8147,2004-02-18,1056.16,59.93,2,8,never,happy,1.94,2.29
2,Cq3823,2007-03-25,210.73,-93.94,1,1,never,happy,-0.11,-0.1
3,Hw9428,2013-12-28,1116.48,80.58,3,10,never,surprised,1.27,1.15
4,xZ0360,2003-08-25,1038.30,12.37,6,17,never,happy,1.76,1.76


Definimos funciones para validar esquemas, buscar nulos y validar duplicados en la columna key

In [41]:


def check_schema(df):
    print("\n-----Validating schema for df")
    print(df.info())

def check_null_values(df):
    print("\n-----Checking null values")
    miss_val = df.isnull().sum()
    miss_val_percent = 100 * df.isnull().sum() / len(df)
    miss_val_table = pd.concat([miss_val, miss_val_percent], axis=1)
    miss_val_table_ren_columns = miss_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    miss_val_table_ren_columns = miss_val_table_ren_columns[
        miss_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print (f"Your selected dataframe has {str(df.shape[1])} columns.\n There are {str(miss_val_table_ren_columns.shape[0])} columns that have missing values.")
    print(miss_val_table_ren_columns)

def find_duplicates_in_key(df):
    print("\n-----Searching for duplicated keys in df")
    print(pd_df[['key_1']].groupby('key_1').filter(lambda x: x['key_1'].count() > 1).count())


Execution without parallelism

In [39]:
time_start = time.time()
find_duplicates_in_key(pd_df)
check_schema(pd_df)
check_null_values(pd_df)
time_end = time.time()
print(f'\nAll done! Took {round(time_end - time_start, 2)} seconds')

-----Searching for duplicated keys in df
key_1    2686
dtype: int64
-----Validating schema for df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269339 entries, 0 to 269338
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   key_1    269339 non-null  object        
 1   date_2   269339 non-null  datetime64[ns]
 2   cont_3   269339 non-null  float64       
 3   cont_4   269339 non-null  float64       
 4   disc_5   269339 non-null  int64         
 5   disc_6   269339 non-null  object        
 6   cat_7    269339 non-null  object        
 7   cat_8    269339 non-null  object        
 8   cont_9   269338 non-null  float64       
 9   cont_10  269338 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 20.5+ MB
None
-----Checking null values
Your selected dataframe has 10 columns.
 There are 2 columns that have missing values.
         Missing Values  % of Total Val

Execution using ThreadPoolExecutor




In [40]:
time_start = time.time()
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(3) as executor:
    executor.submit(find_duplicates_in_key(pd_df))
    executor.submit(check_schema(pd_df))
    executor.submit(check_null_values(pd_df))
time_end = time.time()
print(f'\nAll done! Took {round(time_end - time_start, 2)} seconds')

-----Searching for duplicated keys in df
key_1    2686
dtype: int64
-----Validating schema for df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269339 entries, 0 to 269338
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   key_1    269339 non-null  object        
 1   date_2   269339 non-null  datetime64[ns]
 2   cont_3   269339 non-null  float64       
 3   cont_4   269339 non-null  float64       
 4   disc_5   269339 non-null  int64         
 5   disc_6   269339 non-null  object        
 6   cat_7    269339 non-null  object        
 7   cat_8    269339 non-null  object        
 8   cont_9   269338 non-null  float64       
 9   cont_10  269338 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 20.5+ MB
None
-----Checking null values
Your selected dataframe has 10 columns.
 There are 2 columns that have missing values.
         Missing Values  % of Total Val

Execution using ProcessPoolExecutor

In [42]:
time_start = time.time()
from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor(3) as pexecutor:
    pexecutor.submit(find_duplicates_in_key(pd_df))
    pexecutor.submit(check_schema(pd_df))
    pexecutor.submit(check_null_values(pd_df))
time_end = time.time()
print(f'\nAll done! Took {round(time_end - time_start, 2)} seconds')


-----Searching for duplicated keys in df
key_1    2686
dtype: int64

-----Validating schema for df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269339 entries, 0 to 269338
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   key_1    269339 non-null  object        
 1   date_2   269339 non-null  datetime64[ns]
 2   cont_3   269339 non-null  float64       
 3   cont_4   269339 non-null  float64       
 4   disc_5   269339 non-null  int64         
 5   disc_6   269339 non-null  object        
 6   cat_7    269339 non-null  object        
 7   cat_8    269339 non-null  object        
 8   cont_9   269338 non-null  float64       
 9   cont_10  269338 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 20.5+ MB
None

-----Checking null values
Your selected dataframe has 10 columns.
 There are 2 columns that have missing values.
         Missing Values  % of Total 